In [1]:

import numpy as np
import pandas as pd
from scipy.stats import rankdata

matrix = np.array([
    [0.087, 0.084, 0.115, 0.111],
    [0.066, 0.079, 0.111, 0.102],
    [0.044, 0.086, 0.085, 0.088],
    [0.088, 0.086, 0.244, 0.078],
    [0.285, 0.063, 0.122, 0.068],
    [0.040, 0.134, 0.018, 0.144],
    [0.030, 0.096, 0.020, 0.148],
    [0.039, 0.130, 0.112, 0.105],
    [0.109, 0.103, 0.124, 0.089], 
    [0.014, 0.139, 0.040, 0.069]
])

sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.25377794, 0.35076977, 0.39287723, 0.00257506])

def marcoss(matrix, weights, criteria_types, sites):
    # Step 1: Define AI and AAI
    ai = np.where(np.array(criteria_types) == 'max', 
                  np.max(matrix, axis=0), 
                  np.min(matrix, axis=0))
    
    aai = np.where(np.array(criteria_types) == 'max', 
                   np.min(matrix, axis=0), 
                   np.max(matrix, axis=0))
    
    # Step 2: Extend matrix with AI and AAI
    extended = np.vstack([matrix, ai, aai])

    # Step 3: Normalize extended matrix
    norm_extended = np.zeros_like(extended, dtype=float)
    for j in range(extended.shape[1]):
        if criteria_types[j] == 'max':
            norm_extended[:, j] = extended[:, j] / ai[j]
        else:
            norm_extended[:, j] = aai[j] / extended[:, j]
    
    # Step 4: Apply weights to normalized matrix
    weighted_extended = norm_extended * weights

    # Step 5: Compute S_i, S_AI, S_AAI
    S = weighted_extended.sum(axis=1)
    S_ai, S_aai = S[-2], S[-1]  # Last two rows are AI and AAI
    S_alts = S[:-2]             # Alternatives only

    # Step 6: Compute utility degrees
    K_plus = S_alts / S_ai
    K_minus = S_alts / S_aai

    # Step 7: Final utility function as compromise score
    utility = (K_plus + K_minus) / (
        1 + (1 - K_plus) / K_plus + (1 - K_minus) / K_minus
    )

    # Step 8: Rank alternatives
    ranks = rankdata(-utility, method='min')

    # Step 9: Return results
    return pd.DataFrame({
        'Site': sites,
        'Utility': utility,
        'Rank': ranks
    }).sort_values(by='Rank').reset_index(drop=True)

# Run MARCOS and show result
result_df = marcoss(matrix, weights, criteria_types, sites)
print(result_df)

     Site   Utility  Rank
0   Site4  5.535950     1
1   Site5  3.755681     2
2   Site9  2.886085     3
3   Site8  2.695337     4
4   Site1  1.866679     5
5   Site2  1.502880     6
6  Site10  1.418368     7
7   Site6  1.229138     8
8   Site3  1.149922     9
9   Site7  0.612443    10
